
# Diseño de software para cómputo científico

----

## Unidad 1: Decoradores en Python

### Agenda de la Unidad 1
---

- Orientación a objetos.
- **Decoradores**.

### Decoradores en Python

 - Permiten cambiar el comportamiento de una función (*sin modificarla*)
 - Reusar código fácilmente

<img align="right" width="1000" src="https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fvignette4.wikia.nocookie.net%2Fpowerlisting%2Fimages%2Fd%2Fdf%2FIron-Man-Transform.gif%2Frevision%2Flatest%3Fcb%3D20150514015703&f=1&nofb=1" alt="Fancy Iron Man suitup">

### Todo en Python es un objeto!

Los objetos se definen por su **Identidad**, **Tipo** y **Valor**.


In [4]:
x = 1
print(id(1), type(x))

9752160 <class 'int'>


In [5]:
x.__add__(2)

3

In [ ]:
# Otros objetos
[1, 2, 3]    # listas
5.2          # flotantes
"hola"       # strings

### Funciones
Las funciones también son objetos

In [6]:
def saludar():
    print('Hola!')

In [12]:
id(saludar)

139762571814656

In [13]:
saludar.__name__

'saludar'

In [14]:
hablar = saludar    # asignamos la funcion a otra variable

In [19]:
hablar() # la podemos llamar

Hola!


## Decorador (definición no estricta)

Un decorador es una *función* **d** que recibe como parámetro otra *función* **a** y retorna una nueva *función* **r**.

- **d**: función decoradora
- **a**: función a decorar
- **r**: función decorada

## Código

In [40]:
def d(a):
    def r(*args, **kwargs):
        # comportamiento previo a la ejecución de a
        a(*args, **kwargs)
        # comportamiento posterior a la ejecución de a
    return r


## Código

In [41]:
def d(a):
    def r(*args, **kwargs):
        print("Inicia ejecucion de", a.__name__)
        a(*args, **kwargs)
        print("Fin ejecucion de", a.__name__)
    return r


## Manipulando funciones

In [42]:
def suma(x, y):
    return x + y

In [43]:
suma(1, 2)

3

In [44]:
suma_decorada = d(suma)
suma_decorada(1, 2)

Inicia ejecucion de suma
Fin ejecucion de suma


## Azúcar sintáctica
A partir de Python 2.4 se incorporó la notación con  **@**  para decorar funciones.

In [45]:
def suma(x, y):
    return x + y

suma = d(suma)

In [46]:
@d
def suma(x, y):
    return x + y

## Ejemplo: Medir el tiempo de ejecución

In [61]:
import time

def timer(method):
    '''Decorator to time a function runtime'''
    def wrapper(*args, **kwargs):
        
        t0 = time.time()
        output = method(*args, **kwargs)
        dt = time.time() - t0
        
        print(f'<{method.__name__} finished in {dt} seconds>')
        return output
    
    return wrapper

In [70]:
@timer
def fib(n):
    values = [0, 1]
    while values[-2] < n:
        values.append(values[-2] + values[-1])
        #time.sleep(1)
    return values

In [76]:
lista = fib(1000)

<fib finished in 2.86102294921875e-06 seconds>


## Decoradores encadenados
Similar al concepto matemático de componer funciones.

In [ ]:
@registrar_uso
@medir_tiempo_ejecucion
def mi_funcion(algunos, argumentos):
    # cuerpo de la funcion

In [ ]:
def mi_funcion(algunos, argumentos):
    # cuerpo de la funcion
    
mi_funcion = registrar_uso(medir_tiempo_ejecucion(mi_funcion))

## Decoradores con parámetros?
 - Se denominan *Fábrica de decoradores*
 - Permiten tener decoradores más flexibles.
 - Ejemplo: un decorador que fuerce el tipo de retorno de una función

In [79]:
def to_string(user_function):
    def inner(*args, **kwargs):
        r = user_function(*args, **kwargs)
        return str(r)
    return inner

In [81]:
@to_string
def count():
    return 42
count()

'42'

## Envolvemos el decorador en una función externa 

In [84]:
def typer(t):
    def decorator(user_function):
        def inner(*args, **kwargs):
            r = user_function(*args, **kwargs)
            return t(r)
        return inner
    return decorator

In [85]:
@typer(str)
def count():
    return 42
count()

'42'

In [91]:
@typer(int)
def edad():
    return 25.5
edad()

25

## Clases decoradoras
 - Decoradores con estado
 - Código mejor organizado

In [113]:
class Decorador(object):
    def __init__(self, a):
        self.a = a
        self.variable = None
        
    def __call__(self, *args, **kwargs):
        # comportamiento previo a la ejecución de a
        r = self.a(*args, **kwargs)
        # comportamiento posterior a la ejecución de a
        return r

In [114]:
@Decorador
def edad():
    return 25.5

edad()   # se ejecuta el método __call__

25.5

## Decoradores (definición más estricta)
Un decorador es un *callable* **d** que recibe como parámetro un *objeto* **a** y retorna un nuevo *objeto* **r** (por lo general del mismo tipo que el orginal o con su misma interfaz).

- **d**: clase que defina el método <code>\_\_call\_\_</code>
- **a**: cualquier objeto
- **r**: objeto decorado

## Decorar clases
Identidad

In [116]:
def identidad(C):
    return C

In [122]:
@identidad
class A:
    pass

A()

## Decorar clases
Cambiar totalmente el comportamiento de una clase

In [123]:
def corromper(C):
    return "hola"

In [124]:
@corromper
class A:
    pass

A()

TypeError: 'str' object is not callable

## Decorar clases
Reemplazar con una nueva clase

In [128]:
def reemplazar_con_X(C):
    class X():
        pass
    return X

In [129]:
@reemplazar_con_X
class MiClase():
    pass

MiClase

__main__.reemplazar_con_X.<locals>.X

## Algunos decoradores comunes
La librería estándar de Python incluye <code>classmethod</code>, <code>staticmethod</code>, <code>property</code>

In [5]:
class Student:

    def __init__(self, first_name, last_name):
        self.first_name = first_name.capitalize()
        self.last_name = last_name.capitalize()

    @property
    def name(self):
        full_name = ' '.join([self.first_name, self.last_name])
        return full_name
    
    @classmethod
    def from_string(cls, name_str):
        first_name, last_name = map(str, name_str.split())
        student = cls(first_name, last_name)
        return student

    @staticmethod
    def is_full_name(name_str):
        names = name_str.split(' ')
        return len(names) > 1

In [15]:
s = Student('maria', 'perez')
s.name

'Maria Perez'

In [19]:
Student.from_string('José Gonzalez')

In [20]:
Student.is_full_name('JoséGonzalez')

False